# Chapter 21: 自訂例外與 raise - 講義

## Part I: 理論基礎

### 章節概覽

**學習目標**：
- 掌握 raise 關鍵字的三種用法
- 能夠設計並實作自訂例外類別
- 理解例外鏈（exception chaining）的機制
- 區分 assert 與 raise 的使用場景

**先備知識**：
- Chapter 20: 例外處理基礎（try-except）
- Chapter 16: 類別與繼承

**預計時長**：60 分鐘

---

### 核心概念

#### 為什麼需要主動拋出例外？

在 Chapter 20，我們學習了如何**捕獲**例外。但在實際開發中，我們也需要**主動拋出**例外：

1. **輸入驗證失敗**：檢查到無效參數時
2. **業務規則違反**：不符合業務邏輯時
3. **狀態檢查失敗**：物件處於無效狀態時
4. **重新拋出例外**：在處理例外後，需要向上傳播

#### 為什麼需要自訂例外？

**內建例外的限制**：
- 語意不夠明確（ValueError 太泛用）
- 無法攜帶業務資訊
- 難以分類處理

**自訂例外的優勢**：
- **語意清晰**：`InsufficientFundsError` 比 `ValueError` 更明確
- **資訊豐富**：可添加帳戶 ID、餘額等屬性
- **分層處理**：可建立例外階層，選擇性捕獲

---

## Part II: 實作演練

### 範例 1: raise 主動拋出例外（基礎）

In [ ]:
# 範例 1: 基本的 raise 用法

def divide(a, b):
    """除法運算，檢查除數為 0 的情況"""
    if b == 0:
        raise ValueError("除數不能為 0")
    return a / b

# 測試正常情況
print("10 / 2 =", divide(10, 2))

# 測試例外情況
try:
    result = divide(10, 0)
except ValueError as e:
    print(f"捕獲到例外：{e}")

In [ ]:
# 範例 1-2: 多重驗證

def withdraw(balance, amount):
    """提款驗證"""
    # 驗證 1: 金額不能為負數
    if amount < 0:
        raise ValueError("提款金額不能為負數")
    
    # 驗證 2: 餘額不足
    if amount > balance:
        raise ValueError("餘額不足")
    
    return balance - amount

# 測試各種情況
try:
    print(withdraw(1000, -100))  # 負數金額
except ValueError as e:
    print(f"錯誤：{e}")

try:
    print(withdraw(1000, 2000))  # 餘額不足
except ValueError as e:
    print(f"錯誤：{e}")

# 正常提款
new_balance = withdraw(1000, 300)
print(f"提款成功，新餘額：{new_balance}")

**範例 1 重點**：
- 使用 `raise` 關鍵字拋出例外
- 語法：`raise ExceptionType("錯誤訊息")`
- 常見內建例外：ValueError、TypeError、IndexError

**問題**：上面的例子中，「餘額不足」也用 ValueError，是否夠明確？

→ 這正是自訂例外的用途！

---

### 範例 2: 自訂 Exception 類別（核心）

In [ ]:
# 範例 2-1: 最小自訂例外

class ValidationError(Exception):
    """資料驗證失敗時拋出"""
    pass

def validate_age(age):
    """驗證年齡"""
    if age < 0:
        raise ValidationError("年齡不能為負數")
    if age > 150:
        raise ValidationError("年齡不合理")
    return True

# 測試
try:
    validate_age(200)
except ValidationError as e:
    print(f"驗證失敗：{e}")
    print(f"例外類型：{type(e).__name__}")

In [ ]:
# 範例 2-2: 攜帶資訊的自訂例外

class InsufficientFundsError(Exception):
    """餘額不足錯誤"""
    
    def __init__(self, account_id, balance, amount):
        self.account_id = account_id
        self.balance = balance
        self.amount = amount
        
        # 建立詳細的錯誤訊息
        message = (
            f"帳戶 {account_id} 餘額不足："
            f"需要 ${amount}，但只有 ${balance}"
        )
        super().__init__(message)

def withdraw_v2(account_id, balance, amount):
    """提款 v2：使用自訂例外"""
    if amount > balance:
        raise InsufficientFundsError(account_id, balance, amount)
    return balance - amount

# 測試
try:
    withdraw_v2("A001", 1000, 2000)
except InsufficientFundsError as e:
    print(f"錯誤：{e}")
    print(f"帳戶 ID：{e.account_id}")
    print(f"當前餘額：${e.balance}")
    print(f"欲提金額：${e.amount}")
    print(f"不足金額：${e.amount - e.balance}")

**範例 2 重點**：
- 自訂例外必須繼承 `Exception`
- 最簡單：只需 `pass`，繼承父類行為
- 進階：定義 `__init__`，添加自訂屬性
- 記得呼叫 `super().__init__(message)` 設定錯誤訊息

**命名規範**：
- ✅ 以 `Error` 或 `Exception` 結尾
- ✅ 使用 PascalCase（大駝峰）
- ✅ 描述性名稱：`InvalidEmailError` 而非 `Error1`

---

### 範例 3: 例外鏈 raise...from（進階）

In [ ]:
# 範例 3-1: 沒有例外鏈（遺失資訊）

import json

class DataError(Exception):
    """資料處理錯誤"""
    pass

def parse_config_bad(text):
    """解析設定（沒有例外鏈）"""
    try:
        data = json.loads(text)
        return data
    except json.JSONDecodeError:
        # 問題：遺失了原始 JSON 錯誤的詳細資訊
        raise DataError("無法解析資料")

# 測試
try:
    parse_config_bad('{invalid json}')
except DataError as e:
    print(f"錯誤：{e}")
    print(f"原始錯誤：{e.__cause__}")  # None

In [ ]:
# 範例 3-2: 使用例外鏈（保留完整資訊）

def parse_config_good(text):
    """解析設定（使用例外鏈）"""
    try:
        data = json.loads(text)
        return data
    except json.JSONDecodeError as e:
        # 使用 from 保留原始錯誤
        raise DataError("無法解析資料") from e

# 測試
try:
    parse_config_good('{invalid json}')
except DataError as e:
    print(f"錯誤：{e}")
    print(f"原始錯誤類型：{type(e.__cause__).__name__}")
    print(f"原始錯誤訊息：{e.__cause__}")

In [ ]:
# 範例 3-3: 例外鏈的三種形式

print("=== 1. 顯式鏈（raise...from）===")
try:
    try:
        int("abc")
    except ValueError as e:
        raise TypeError("轉換失敗") from e
except TypeError as e:
    print(f"__cause__: {e.__cause__}")  # 有值
    print(f"__context__: {e.__context__}")  # 也有值

print("\n=== 2. 隱式鏈（自動）===")
try:
    try:
        int("abc")
    except ValueError:
        raise TypeError("轉換失敗")  # 沒有 from
except TypeError as e:
    print(f"__cause__: {e.__cause__}")  # None
    print(f"__context__: {e.__context__}")  # 有值

print("\n=== 3. 抑制鏈（from None）===")
try:
    try:
        int("abc")
    except ValueError:
        raise TypeError("轉換失敗") from None
except TypeError as e:
    print(f"__cause__: {e.__cause__}")  # None
    print(f"__context__: {e.__context__}")  # None

**範例 3 重點**：
- **顯式鏈**：`raise NewError() from original_error`
  - 設定 `__cause__` 屬性
  - 用於：包裝低階錯誤為高階錯誤
- **隱式鏈**：`raise NewError()`（在 except 中）
  - 自動設定 `__context__` 屬性
  - Python 自動追蹤
- **抑制鏈**：`raise NewError() from None`
  - 隱藏原始錯誤
  - 用於：不想暴露內部實作細節時

---

### 範例 4: assert 斷言（實務）

In [ ]:
# 範例 4-1: assert 基本用法

def calculate_average(numbers):
    """計算平均值（使用 assert 檢查）"""
    # assert 用於檢查內部假設
    assert len(numbers) > 0, "內部錯誤：不應傳入空列表"
    assert all(isinstance(n, (int, float)) for n in numbers), "數字類型錯誤"
    
    return sum(numbers) / len(numbers)

# 測試
print("平均值：", calculate_average([10, 20, 30]))

try:
    calculate_average([])  # 觸發 AssertionError
except AssertionError as e:
    print(f"斷言失敗：{e}")

In [ ]:
# 範例 4-2: assert vs raise 的比較

# ❌ 錯誤：用 assert 驗證使用者輸入
def withdraw_bad(amount):
    assert amount > 0, "金額必須為正數"
    # 問題：python -O 執行時，assert 會被忽略！
    return amount

# ✅ 正確：用 raise 驗證使用者輸入
def withdraw_good(amount):
    if amount <= 0:
        raise ValueError("金額必須為正數")
    return amount

# 演示 assert 的特性
print("=== assert 用於內部檢查 ===")
def _internal_helper(data):
    # 這是內部函式，不對外公開
    # 假設呼叫方已驗證過 data
    assert data is not None, "內部錯誤：data 不應為 None"
    return len(data)

print("=== raise 用於外部驗證 ===")
def public_api(data):
    # 這是公開 API，需要驗證使用者輸入
    if data is None:
        raise ValueError("data 不能為 None")
    return _internal_helper(data)

**範例 4 重點**：

| 特性 | assert | raise |
|:-----|:-------|:------|
| **用途** | 除錯檢查 | 錯誤處理 |
| **執行** | 可被停用（`-O`） | 永遠執行 |
| **例外** | AssertionError | 任意例外 |
| **場景** | 內部假設 | 外部輸入 |

**使用原則**：
- ✅ assert：檢查「不應該發生」的情況（程式邏輯錯誤）
- ✅ raise：處理「可能發生」的情況（使用者錯誤、資料錯誤）

---

### 範例 5: 完整自訂例外系統（整合）

In [ ]:
# 範例 5: 銀行系統例外階層

# 第 1 層：基底例外
class BankError(Exception):
    """銀行系統所有例外的基底類別"""
    pass

# 第 2 層：分類例外
class AccountError(BankError):
    """帳戶相關錯誤"""
    pass

class TransactionError(BankError):
    """交易相關錯誤"""
    pass

# 第 3 層：具體例外
class InvalidAccountError(AccountError):
    """帳戶不存在或無效"""
    def __init__(self, account_id):
        self.account_id = account_id
        super().__init__(f"帳戶 {account_id} 不存在")

class AccountLockedError(AccountError):
    """帳戶已被鎖定"""
    def __init__(self, account_id, reason):
        self.account_id = account_id
        self.reason = reason
        super().__init__(f"帳戶 {account_id} 已鎖定：{reason}")

class InsufficientFundsError(TransactionError):
    """餘額不足"""
    def __init__(self, account_id, balance, amount):
        self.account_id = account_id
        self.balance = balance
        self.amount = amount
        super().__init__(
            f"帳戶 {account_id} 餘額不足：需要 ${amount}，但只有 ${balance}"
        )

class DailyLimitExceededError(TransactionError):
    """超過每日限額"""
    def __init__(self, account_id, daily_limit, total_today):
        self.account_id = account_id
        self.daily_limit = daily_limit
        self.total_today = total_today
        super().__init__(
            f"超過每日限額：已使用 ${total_today}/{daily_limit}"
        )

# 使用範例
class BankAccount:
    """銀行帳戶類別"""
    
    def __init__(self, account_id, balance=0, daily_limit=10000):
        self.account_id = account_id
        self.balance = balance
        self.daily_limit = daily_limit
        self.today_total = 0
        self.is_locked = False
        self.lock_reason = None
    
    def withdraw(self, amount):
        """提款"""
        # 驗證 1: 帳戶狀態
        if self.is_locked:
            raise AccountLockedError(self.account_id, self.lock_reason)
        
        # 驗證 2: 餘額
        if amount > self.balance:
            raise InsufficientFundsError(self.account_id, self.balance, amount)
        
        # 驗證 3: 每日限額
        if self.today_total + amount > self.daily_limit:
            raise DailyLimitExceededError(
                self.account_id, self.daily_limit, self.today_total
            )
        
        # 執行提款
        self.balance -= amount
        self.today_total += amount
        return amount
    
    def lock(self, reason):
        """鎖定帳戶"""
        self.is_locked = True
        self.lock_reason = reason

# 測試不同的例外情況
account = BankAccount("A001", balance=5000, daily_limit=3000)

print("=== 測試 1: 正常提款 ===")
try:
    amount = account.withdraw(1000)
    print(f"提款成功：${amount}，餘額：${account.balance}")
except BankError as e:
    print(f"錯誤：{e}")

print("\n=== 測試 2: 超過每日限額 ===")
try:
    account.withdraw(2500)  # 今日已提 1000，再提 2500 超過 3000
except DailyLimitExceededError as e:
    print(f"錯誤：{e}")
    print(f"已使用額度：${e.total_today}")
except BankError as e:
    print(f"其他錯誤：{e}")

print("\n=== 測試 3: 帳戶鎖定 ===")
account.lock("異常交易")
try:
    account.withdraw(500)
except AccountLockedError as e:
    print(f"錯誤：{e}")
    print(f"鎖定原因：{e.reason}")
except BankError as e:
    print(f"其他錯誤：{e}")

print("\n=== 測試 4: 分層捕獲 ===")
account2 = BankAccount("A002", balance=100)
try:
    account2.withdraw(500)
except InsufficientFundsError as e:
    # 特定處理
    print(f"[特定] 餘額不足：缺 ${e.amount - e.balance}")
except TransactionError as e:
    # 交易類錯誤
    print(f"[分類] 交易錯誤：{e}")
except BankError as e:
    # 所有銀行錯誤
    print(f"[通用] 銀行錯誤：{e}")

**範例 5 重點**：

**例外階層設計原則**：
1. **基底例外**：所有自訂例外的共同祖先（如 `BankError`）
2. **分類例外**：中間層，用於分組（如 `AccountError`, `TransactionError`）
3. **具體例外**：最底層，具體錯誤（如 `InsufficientFundsError`）

**分層捕獲的優勢**：
```python
try:
    # 銀行操作
except InsufficientFundsError:  # 最具體
    # 特殊處理餘額不足
except TransactionError:  # 中等
    # 處理所有交易錯誤
except BankError:  # 最通用
    # 處理所有銀行錯誤
```

**每個例外都攜帶資訊**：
- 帳戶 ID、餘額、金額等
- 方便除錯和日誌記錄
- 可用於錯誤恢復

---

## Part III: 本章總結

### 知識回顧

#### 1. raise 的三種用法
```python
# 用法 1: 拋出新例外
raise ValueError("錯誤訊息")

# 用法 2: 重新拋出（在 except 中）
try:
    risky()
except Exception:
    log_error()
    raise  # 重新拋出

# 用法 3: 例外鏈
try:
    low_level()
except LowError as e:
    raise HighError() from e
```

#### 2. 自訂例外模板
```python
class MyError(Exception):
    """描述例外用途"""
    
    def __init__(self, arg1, arg2):
        self.arg1 = arg1
        self.arg2 = arg2
        message = f"錯誤描述：{arg1}, {arg2}"
        super().__init__(message)
```

#### 3. 例外階層設計
```
BaseError (基底)
├── CategoryError1 (分類)
│   ├── SpecificError1A
│   └── SpecificError1B
└── CategoryError2
    └── SpecificError2A
```

#### 4. assert vs raise
- **assert**：檢查內部邏輯假設，可被停用
- **raise**：處理實際錯誤，永遠執行

---

### 常見誤區

#### 誤區 1: 過度使用內建例外
❌ **錯誤**：
```python
if balance < amount:
    raise ValueError("餘額不足")
if card.expired:
    raise ValueError("卡片過期")
# 都用 ValueError，無法區分
```

✅ **正確**：
```python
if balance < amount:
    raise InsufficientFundsError(balance, amount)
if card.expired:
    raise CardExpiredError(card)
# 類別名稱即文件
```

#### 誤區 2: 忽略例外鏈
❌ **錯誤**：
```python
try:
    json.loads(text)
except JSONDecodeError:
    raise DataError("解析失敗")  # 遺失原始錯誤
```

✅ **正確**：
```python
try:
    json.loads(text)
except JSONDecodeError as e:
    raise DataError("解析失敗") from e  # 保留鏈
```

#### 誤區 3: 用 assert 驗證輸入
❌ **錯誤**：
```python
def api(data):
    assert data is not None  # 會被 -O 停用！
```

✅ **正確**：
```python
def api(data):
    if data is None:
        raise ValueError("data 不能為 None")
```

---

### 自我檢核

完成本章後，您應該能夠：

- [ ] 使用 `raise` 拋出內建例外
- [ ] 創建自訂例外類別
- [ ] 為例外添加自訂屬性
- [ ] 使用 `raise...from` 建立例外鏈
- [ ] 設計多層次的例外階層
- [ ] 區分 `assert` 與 `raise` 的使用時機
- [ ] 分層捕獲例外（from specific to general）
- [ ] 理解 `__cause__` 與 `__context__` 的差異

---

### 延伸閱讀

1. **Python 官方文件**
   - [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)
   - [Exception Hierarchy](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)

2. **PEP 文件**
   - PEP 3134: Exception Chaining and Embedded Tracebacks

3. **最佳實踐**
   - Effective Python, Item 87: Define a Root Exception
   - Clean Code, Chapter 7: Error Handling

---

### 下一步

完成 `02-worked-examples.ipynb` 的實務案例練習，深入理解：
- 表單驗證系統
- 支付處理系統
- 遊戲狀態系統

**學習提醒**：好的例外設計是專業程式的標誌！